In [ ]:
import sys
import os

current_notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root_dir = os.path.abspath(os.path.join(current_notebook_dir, '../../'))

# 将这个父目录添加到sys.path的最前面
if project_root_dir not in sys.path:
    sys.path.insert(0, project_root_dir)

print(sys.path)

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from pytorch_script.visual_utils import load_cifar10_data, \
	get_sorted_model_paths, evaluate_model_performance, load_model_state_dict


In [ ]:
import seaborn as sns

config = {
	'model_dir': '../../model_training_results/cifar10_resnet20',  # 模型存储的文件夹路径
	'model_prefix': 'model_',        # 模型文件名的前缀
	'model_extension': '.pth',                         # 保存模型的文件扩展名
	'cifar10_data_path': '../../pytorch_script/data/cifar10',           # CIFAR-10 数据集存储路径
	'batch_size': 64,                                 # DataLoader 的批次大小
	'num_workers': 2                                  # DataLoader 的工作进程数
}

device = torch.device("cuda")

# 加载数据
train_loader, test_loader = load_cifar10_data(
	config['cifar10_data_path'], config['batch_size'], config['num_workers']
)
if train_loader is None or test_loader is None:
	print("数据加载失败，程序终止。")

# 获取排序后的模型文件路径
sorted_model_paths = get_sorted_model_paths(
	config['model_dir'], config['model_prefix'], config['model_extension']
)

if not sorted_model_paths:
	print("没有找到符合条件或能成功加载的模型文件。请检查 MODEL_DIR, MODEL_PREFIX 和 MODEL_EXTENSION 设置。")

epochs = []
train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

print("\n开始逐个评估模型...")
for epoch, model_path in sorted_model_paths:
	print(f"\n正在评估模型：{model_path} (纪元: {epoch})")
	
	# 评估训练集性能
	model = load_model_state_dict('cifar10', 'resnet20', 10, model_path, device)
	train_acc, train_loss = evaluate_model_performance(model, train_loader, device)
	if train_acc is not None:
		train_accuracies.append(train_acc)
		train_losses.append(train_loss)
		print(f"  training acc = {train_acc:.2f}%")
		print(f"  training loss = {train_loss:.2f}")
	else:
		print("  训练集评估失败，跳过。")
		continue # 如果训练集评估失败，则整个模型跳过

	# 评估测试集性能
	test_acc, test_loss = evaluate_model_performance(model, test_loader, device)
	if test_acc is not None:
		test_accuracies.append(test_acc)
		test_losses.append(test_loss)
		print(f"  test acc = {test_acc:.2f}%")
		print(f"  test loss = {train_loss:.2f}")
	else:
		print("  测试集评估失败，跳过。")
		continue # 如果测试集评估失败，则整个模型跳过

	epochs.append(epoch)

In [ ]:
np.save('train_accuracies.npy', np.array(train_accuracies))
np.save('test_accuracies.npy', np.array(test_accuracies))
np.save('train_losses.npy', np.array(train_losses))
np.save('test_losses.npy', np.array(test_losses))

In [ ]:
def model_losses(model, data_loader):
  losses = []
  model.eval()
  for images, labels in data_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = torch.nn.functional.cross_entropy(outputs, labels)
    losses.append(loss.item())
  return losses

In [ ]:
all_model_train_losses = {}
all_model_test_losses = {}

for epoch, model_path in sorted_model_paths:
	print(epoch)
	model = load_model_state_dict('cifar10', 'resnet20', 10, model_path, device)
	model.to(device)
	all_model_train_losses[epoch] = model_losses(model, train_loader)
	all_model_test_losses[epoch] = model_losses(model, test_loader)

In [11]:
import pickle
file_path_pickle = "all_model_train_losses.pickle"
with open(file_path_pickle, 'wb') as f: # 注意 'wb' 表示写入二进制
    pickle.dump(all_model_train_losses, f)

file_path_pickle = "all_model_test_losses.pickle"
with open(file_path_pickle, 'wb') as f: # 注意 'wb' 表示写入二进制
    pickle.dump(all_model_test_losses, f)

In [12]:
np.save('epochs.npy', epochs)